In [1]:
import requests
import pandas as pd
import subprocess as sp
import sys
from glob import glob
import re
from pysam import VariantFile, VariantRecord
from collections import defaultdict
import csv 


In [2]:
"""
Download the latest files from the mutation catalogue github repo

The excel file contains the graded mutations and their confidence levels
The vcf file contains the genomic coordinates of the mutations linked to the graded mutations
"""

excel_file_url = 'https://github.com/GTB-tbsequencing/mutation-catalogue-2023/raw/main/Final%20Result%20Files/WHO-UCN-TB-2023.5-eng.xlsx'
r = requests.get(excel_file_url, allow_redirects=True)
excel_file = 'WHO-UCN-TB-2023.5-eng.xlsx'
open(excel_file, 'wb').write(r.content)

vcf_file_url = 'https://github.com/GTB-tbsequencing/mutation-catalogue-2023/raw/main/Final%20Result%20Files/Genomic%20coordinates_7Sep2023.vcf.gz'
r = requests.get(vcf_file_url, allow_redirects=True)
open('Genomic_coordinates_7Sep2023.vcf.gz', 'wb').write(r.content)

789313

In [3]:
"""
snpEff doesn't use the right codon table for Mtb, so we need to modify the config file
"""
snpeff_dirs = glob(f'{sys.prefix}/share/snpeff*')
if len(snpeff_dirs)==1:
    snpeff_dir = snpeff_dirs[0]
    snpeff_config = f'{snpeff_dir}/snpEff.config'
    if "Mycobacterium_tuberculosis_h37rv.Chromosome.codonTable : Bacterial_and_Plant_Plastid" not in open(snpeff_config).read():
        open(snpeff_config, 'a').write("\nMycobacterium_tuberculosis_h37rv.Chromosome.codonTable : Bacterial_and_Plant_Plastid\n")
    else:
        print("Right codon table alread set")
else:
    print("Multiple or no snpeff installations found. Please remove all but one.")

Right codon table alread set


In [4]:
"""
We annotate the vcf file with snpEff to make sure we are getting the same annotations as the graded mutations
"""
sp.call("bcftools view Genomic_coordinates_7Sep2023.vcf.gz | sed 's/NC_000962.3/Chromosome/' | snpEff ann Mycobacterium_tuberculosis_h37rv  > ann.vcf", shell=True)

0

In [6]:

"""
This function extracts the both the graded mutation and the snpEff mutation from the vcf file
"""
def extract_mutation(var: VariantRecord) -> dict:
    nucleotide_types = set(['initiator_codon_variant','synonymous_variant','upstream_gene_variant','splice_region_variant&stop_retained_variant','splice_region_variant&non_coding_transcript_exon_variant','non_coding_transcript_exon_variant'])
    gene = var.info['graded_variant'].split("_")[0]
    cat_mut = '_'.join(var.info['graded_variant'].split("_")[1:])
    annotations = []
    for a in var.info['ANN']:
        a = a.split('|')
        if a[1] in ('downstream_gene_variant'):
            continue
        if a[3]==gene:
            annotations.append(a)
    if len(annotations) == 0:
        raise Exception
    else:
        a = annotations[0]
        if a[1] in nucleotide_types:
            mutation = a[9]
        elif a[1]=='start_lost':
            mutation = 'p.Met1?'
        elif r:=re.match('p.Ter(\d+)fs',a[10]):
            mutation = f'p.Ter{r.group(1)}ext*?'
        else:
            mutation = a[10]
        return {
            'gene':gene,
            'catalogue_mutation':cat_mut,
            'snpEff_mutation':mutation,
            'type': a[1]
        }


In [7]:
"""
Process the vcf file and compare the graded mutations with the snpEff mutations
Some of the mutations don't agree in terms of the assigned graded mutation and the mutation from snpEff so we dump those
"""
processed = []
dump = []
vcf = VariantFile('ann.vcf')
for i,var in enumerate(vcf):
    try:
        obj = extract_mutation(var)
    except:
        break
    if obj['catalogue_mutation']!=obj['snpEff_mutation']:
        dump.append((var,obj))
    else:
        processed.append(obj)


print('Number of validated mutations:', len(processed))
print('Number of mutations that could not be parsed:', len(dump))





Number of validated mutations: 76779
Number of mutations that could not be parsed: 40944


In [8]:
"""
Create a lookup table for the mutations graded mutation -> snpEff mutation
This is a one to many relationship. For example the graded mutation katG LoF is linked to many snpEff mutations (e.g. frameshifts)
"""
variant_lookup = defaultdict(set)
for var in processed:
    key = (var['gene'],var['catalogue_mutation'])
    val = (var['gene'],var['snpEff_mutation'])
    variant_lookup[key].add(val)

In [9]:
""""
Here we run through the excel file and lookup the graded mutations in the lookup table.
If the mutation is not found in the lookup table we dump it to a json file for checking.
If all goes well they should all be found in the lookup table.
"""

unparsed = []
lof_variants = ['deletion','LoF']
confidence = {}
rows = []
gene_drug_associations = set()
for row in pd.read_excel(excel_file,sheet_name='Catalogue_master_file', skiprows=2).to_dict('records'):
    if row['gene']=='dnaA' and row['mutation'].startswith('c.-'): 
        continue
    drug = row['drug'].lower()
    key = (drug,row['gene'],row['mutation'])
    vkey = (row['gene'],row['mutation'])
    if row['mutation'] not in lof_variants and vkey not in variant_lookup:
        unparsed.append(key)
        continue
    gene_drug_associations.add((row['gene'], drug))
    confidence = row['FINAL CONFIDENCE GRADING'][3:]
    for v in variant_lookup[vkey]:
        r = {
            'Gene':v[0],
            'Mutation':v[1],
            'type': 'drug_resistance' if confidence in ('Assoc w R', 'Assoc w R - Interim') else 'who_confidence',
            'drug': drug,
            'original_mutation': row["mutation"],
            'confidence': confidence,
            'comment': row["Comment"]
        }
        rows.append(r)
    if row['mutation'] in lof_variants:
        if row['mutation']=='deletion':
            so_terms = ['feature_ablation']
        elif row['mutation']=='LoF':
            so_terms = ['feature_ablation','start_lost','stop_gained','frameshift_variant']
        for so_term in so_terms:
            r = {
                'Gene':row['gene'],
                'Mutation': so_term,
                'type': 'drug_resistance' if confidence in ('Assoc w R', 'Assoc w R - Interim') else 'who_confidence',
                'drug': drug,
                'original_mutation': row["mutation"],
                'confidence': confidence,
                'comment': row["Comment"]
            }
            rows.append(r)

print("Number of unparsed mutations:",len(unparsed))
import json
json.dump(unparsed,open('who.json','w'),indent=4)

/Users/jody/mambaforge/envs/pp/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Number of unparsed mutations: 0


In [ ]:
"""
Create the mutations output file 
"""

import pandas as pd
df = pd.DataFrame(rows)
df.to_csv('parsed_who_mutations.csv',index=False)

In [ ]:
"""
Create a watchlist with all genes, regardless if they have a tier 1/2 mutation or not
"""

watchlist_rows = []
for gene,drug in gene_drug_associations:
    watchlist_rows.append({
        'Gene':gene,
        'Type':'drug_resistance',
        'Drug':drug,
    })
with open("genes.csv","w")  as O:
    writer = csv.DictWriter(O,fieldnames=['Gene','Type','Drug'])
    writer.writeheader()
    writer.writerows(watchlist_rows)